In [5]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [279]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.0-turbo'

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool

In [ ]:
# Initialize the tool with the website URL, so the agent can only scrap the content of the specified website
docs_scrape_tool = ScrapeWebsiteTool(
    url=input("Enter website URL: "), #"https://www.aicamp.ai/event/eventdetails/W2025010100"
    website_url = url 
)

## Creating and run the Crew

Create your crew of Agents
Pass the tasks to be performed by those agents.
Note: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.
verbose=2 allows you to see all the logs of the execution.

In [ ]:
def main():
    #Creat Agent
    event_agent = Agent(
        role="Event Content Researcher",
        goal="verify whether {text} is about an event information "
            "collect the {text} as event_content and event url if it is event information"
            "store the event data in JSON format", 
        backstory=(
            "You need to verify the input information about events and"
            "Check that the URL of the event is valid."
            "Make sure to provide event data in the required format, "
            " and make no assumptions."
        ),
        allow_delegation=False,
        verbose=True
    )    


    #creat task
    webscraping_agent = Task(
        description=(
            "If the {text} is not about an event information, ask user to input the event URL again."
            "If the {text} is about an event information,"
			"You need to scrap the event content on the web via the webpage per event URL."
            "Be sure to provide the event data including title, date, organization, leader or speaker or instructor, URL, and content."
            "You must store the event data in the required format"
            "and accurate response to the user without assumption."
        ),
        expected_output=(
            "The event content from the webpage should be provided."
            "The event data is in JSON format."
        ),
        tools=[docs_scrape_tool],
        agent=event_agent
    )
    
    
    #creat the crew
    crew = Crew(
        agents=[event_agent],
        tasks=[webscraping_agent]   
    )
    
    result = crew.kickoff()
    #print(result)
    
    return result

if __name__ == "__main__":
  result = main()